# **Installing libaries**

In [1]:
!pip install newspaper3k
!pip install GoogleNews

     |████████████████████████████████| 215kB 4.2MB/s 
     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 7.4MB 6.3MB/s 
     |████████████████████████████████| 194kB 37.9MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp36-none-any.whl size=13538 sha256=1e4d4cd80d71a6fb2f64f8201a706f296fa6f4256e24b0b2257b5f6c4bac892c
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl size=3355 sha256=c1cf7d8af0bbb3bfa8a1ce5a9b24784ba5af8dc7fa6c316bb3093c4925bdaa12
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398406 sha256=f25c4dac7b04069d7186f46ac251888c34b16bc1d260399ffef19364d1b6e850
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35

# ***Importing libaries***

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from GoogleNews import GoogleNews
from newspaper import Article
from newspaper import Config
nltk.download('punkt')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# **Code**
In this notbook I'm automating google news. [Google News](https://news.google.com/)
### Requirements to run this code:

*   Google Colab GPU otherwise its give runtime error.
*   User agent (Everyone has different) [Here is Link to search your own.](https://www.whatismybrowser.com/detect/what-is-my-user-agent)
*   Search words list
*   Strating and Ending Date



In [3]:
#config will allow us to access the specified url for which we are #not authorized. 
#Sometimes we may get 403 client error while parsing the link to download the article.
search_words = ['child labour in pakistan', 'child abuse in pakistan', 'rape case in pakistan', 'child harassment in pakistan', "forced to girl in pakistan", 'sexual harassment with girl in Pakistan']
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36' #search your user agent
config = Config()
config.browser_user_agent = user_agent
googlenews=GoogleNews(start='05/01/2018',end='08/27/2020')
for word in search_words:
    googlenews.search(word)
    result=googlenews.result()
    df=pd.DataFrame(result)
    for i in range(2,20):
        googlenews.getpage(i)
        result=googlenews.result()
        df=pd.DataFrame(result)
    list=[]
    for ind in df.index:
        dict={}
        article = Article(df['link'][ind],config=config)
        try: #some articles creating problem in (parse) so i use try and except
            article.download()
            article.parse()
            article.nlp()
        except:
            pass
        dict['Date']=df['date'][ind]
        dict['Media']=df['media'][ind]
        author = article.authors
        authors=""
        for word in author: #using for loop beacause author return list [] as i converting into string
          authors+=word+', '
        dict['authors'] = authors
        dict['Title']=article.title
        dict['Article']=article.text
        dict['Summary']=article.summary
        keyword=article.keywords
        keywords=""
        for word in keyword:
          keywords+=word+', '
        dict['keywords'] = keywords
        dict['link'] = article.url
        list.append(dict)  
    news_df=pd.DataFrame(list)

/usr/local/lib/python3.6/dist-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname IST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [7]:
news_df.head()

,Date,Media,authors,Title,Article,Summary,keywords,link
0,"Jun 5, 2020",Reuters,"Zofeen T. Ebrahim, Min Read,",Eight-year-old maid's death spurs calls for ch...,KARACHI (Thomson Reuters Foundation) - The bru...,KARACHI (Thomson Reuters Foundation) - The bru...,"rights, work, child, calls, labour, reform, hu...",https://www.reuters.com/article/us-pakistan-ch...
1,"Jun 6, 2020",OpIndia,"Opindia Staff, Https, Staff Reporter At Opindia,",Pakistan: 8 year old maid beaten to death for ...,"On Sunday, the Pakistan police had reportedly ...",The Ministry had also proposed amendments to t...,"social, old, domestic, following, maid, escape...",https://www.opindia.com/2020/06/pakistan-8-yea...
2,"Mar 26, 2019",The News International,"Myra Imran,",MoHR launches survey on child labour in Pakistan,Islamabad : To collect the scientific data of ...,Islamabad : To collect the scientific data of ...,"shireen, rights, child, labour, secretary, hum...",https://www.thenews.com.pk/print/448859-mohr-l...
3,"Oct 22, 2019",The Express Tribune,,Child labour in Pakistan,Not all children in Pakistan are lucky enough ...,"Though there are laws banning child labour, ch...","provide, work, child, working, labour, need, c...",http://tribune.com.pk/letter/2084376/6-child-l...
4,"Jun 25, 2019",The News International,"Jamila Achakzai,",Report notices Pakistan’s progress against chi...,"Islamabad: As Save the Children, an internatio...","Islamabad: As Save the Children, an internatio...","pakistans, child, labour, society, help, organ...",https://www.thenews.com.pk/print/489426-report...


In [ ]:
news_df.to_excel("articles1.xlsx")